In [42]:
# iimport all the necessary packages
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd

In [43]:
# import os and find the path
import os
print(os.getcwd())

/Users/alexandros.samartzis/Spiced_Academy/stationary-sriracha-student-code/Week_2/spiced_projects


In [44]:
# load data
titanik = pd.read_csv('train.csv', index_col=0)


In [45]:
# Delete unnecessary columns
del titanik['Ticket']
del titanik['Name']
#del titanik['Embarked']
del titanik['Cabin']

In [46]:
# Find in a table the NaN of each Column
titanik.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [47]:
# Find the unique values of the column
titanik['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

## Make a pipeline to Impute the NaN

In [48]:
# create a sequential pipeline for getting rid of the NaN for numerical
numeric_imputer = make_pipeline(
    SimpleImputer(strategy='mean')
    )

In [49]:
# create a sequential pipeline for getting rid of the NaN for numerical
categorical_imputer = make_pipeline(
    SimpleImputer(strategy='most_frequent')
    )

In [50]:
preprocessor_imputer = ColumnTransformer(
    transformers=[
        ("num_imputer", numeric_imputer, ['Age']),
        ("cat_imputer", categorical_imputer, ['Embarked'])
    ],
    remainder='passthrough')

In [51]:
# create the model pipeline
pipeline_imputer = make_pipeline(preprocessor_imputer)

In [52]:
# fit the pipeline to training data
titanik_joker = pipeline_imputer.fit_transform(titanik)

In [53]:
# Create a Data Frame to feed with the previous columns of the initial DF
titanik_transf = pd.DataFrame(titanik_joker, columns=titanik.columns)


In [54]:
titanik_transf.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [55]:
# split to X, and y
X = titanik_transf.iloc[:,1:]
y = titanik_transf['Survived']

In [56]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer_OHE = make_pipeline(
    OneHotEncoder(sparse=False, handle_unknown='ignore')
    )

In [57]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer_BIN = make_pipeline(
    KBinsDiscretizer(n_bins=5, encode='onehot', strategy='quantile')
    )

In [58]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer_SCALE = make_pipeline(
    StandardScaler()
    )

In [59]:
# feature engineering for categorical
categorical_features = ['Embarked',"Sex", "Pclass"]

In [60]:
# handle unkno
categorical_transformer_OHE = OneHotEncoder(handle_unknown="ignore")

In [61]:
# create a sequential pipeline 
# output of one step will be input to the next 
categorical_transformer_BIN = make_pipeline(
    KBinsDiscretizer(n_bins=5, encode='onehot', strategy='quantile')
    )

In [62]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num_OHE", numeric_transformer_OHE, ["Pclass", 'SibSp']),
        ("num_AGE", numeric_transformer_BIN, ["Age"]),
        ("num_SCALE", numeric_transformer_SCALE, ["Age","Parch","Fare"]),
        ("cat", categorical_transformer_OHE, categorical_features)
    ],
    remainder='passthrough')

In [63]:
# create the model pipeline
pipeline = make_pipeline(preprocessor, LogisticRegression(max_iter=300))


In [64]:
# split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

In [65]:
X.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [77]:
titanik_transf['SibSp'].unique()

array(['male', 'female'], dtype=object)

In [ ]:
# fit the pipeline to training data
pipeline.fit(X_train, y_train)

In [ ]:
# calculate the accuracy score from test data
print("model score: %.3f" % pipeline.score(X_test, y_test))

In [ ]:
# get predictions from the pipeline
print(pipeline.predict(X_test))

In [ ]:
# get prediction probabilities from the pipeline 
print(pipeline.predict_proba(X_test)[:, 1])